In [1]:
# Cell 1: Install required packages with specific versions
!pip install transformers==4.28.1 torch pandas scikit-learn wandb

In [2]:
# Cell 2: Import all necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import random

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [23]:
# Cell 3: Load and clean data
def load_and_clean_data(file_path):
    df = pd.read_csv(file_path)

    # Check if the expected columns exist
    required_columns = ['input_name', 'variation', 'target_script', 'label']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("CSV file doesn't contain all required columns")

    # Remove rows where label is not numeric
    df = df[pd.to_numeric(df['label'], errors='coerce').notnull()]

    # Convert label to integer
    df['label'] = df['label'].astype(int)

    # Validate labels are only 0 or 1
    invalid_labels = df[~df['label'].isin([0, 1])]
    if not invalid_labels.empty:
        print(f"Warning: Found {len(invalid_labels)} rows with invalid labels (not 0 or 1). Dropping them.")
        df = df[df['label'].isin([0, 1])]

    # Create text column
    df['text'] = df.apply(lambda x: f"{x['input_name']} [SEP] {x['variation']} ", axis=1)

    print(f"Loaded {len(df)} valid samples")
    print("Label distribution:")
    print(df['label'].value_counts())

    return df

try:
    df = load_and_clean_data('synthetic_name_variations_1500.csv')
    display(df.head())
except Exception as e:
    print(f"Error loading data: {str(e)}")

Loaded 1494 valid samples
Label distribution:
label
1    748
0    746
Name: count, dtype: int64


,input_name,variation,target_script,label,text
0,Adewumi Bakare,أديومي باكار,arab,1,Adewumi Bakare [SEP] أديومي باكار
1,Adewumi Bakare,أدوومي بكر,arab,0,Adewumi Bakare [SEP] أدوومي بكر
2,Chinwe Okonkwo,شينوي أوكونكو,arab,1,Chinwe Okonkwo [SEP] شينوي أوكونكو
3,Chinwe Okonkwo,شينو أكونكو,arab,0,Chinwe Okonkwo [SEP] شينو أكونكو
4,Temitope Adekunle,تيميتوپي أديكونلي,arab,1,Temitope Adekunle [SEP] تيميتوپي أديكونلي


In [24]:
# Cell 4: Dataset class with robust label handling
class NameVariationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        # Robust label handling
        label = self.labels[idx]
        if isinstance(label, (int, float, np.integer)):
            label = int(label)
        elif isinstance(label, str):
            try:
                label = int(float(label))  # Handles both '0' and '0.0'
            except:
                label = 0  # Default to 0 if conversion fails
                print(f"Warning: Could not convert label '{self.labels[idx]}' to integer")

        # Ensure label is 0 or 1
        label = 0 if label not in [0, 1] else label

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [25]:
# Cell 5: Initialize tokenizer and create datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values,
    df['label'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Create datasets
train_dataset = NameVariationDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = NameVariationDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training samples: 1195
Validation samples: 299


In [26]:
# Cell 6: Initialize model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=2,
    ignore_mismatched_sizes=True
)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Model loaded on {device}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification fro

Model loaded on cuda


In [27]:
# Cell 7: Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy='steps',
    eval_steps=200,
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
    report_to="none"  # Disable W&B if not needed
)

In [28]:
# Cell 8: Compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [29]:
# Cell 9: Initialize and run Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("Starting training...")
trainer.train()
print("Training completed!")

/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:645: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Starting training...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,0.375000,0.377825,0.842809,0.852665,0.804734,0.906667


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Training completed!


In [30]:
# Cell 10: Evaluate and save
eval_results = trainer.evaluate()
print("\nEvaluation results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Save model
trainer.save_model("./results")
tokenizer.save_pretrained("./results")
print("\nModel saved to './results' directory")


Evaluation results:
eval_loss: 0.4155
eval_accuracy: 0.8763
eval_f1: 0.8787
eval_precision: 0.8645
eval_recall: 0.8933
eval_runtime: 0.8537
eval_samples_per_second: 350.2490
eval_steps_per_second: 11.7140
epoch: 5.0000

Model saved to './results' directory


In [39]:
# Cell 11: Inference testing samples
def predict(text, model, tokenizer, device):
    """Helper function for making predictions"""
    inputs = tokenizer(
        text,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred].item()

    return pred, confidence

# Test samples covering different scenarios
test_samples = [
    # Positive examples (should predict 1)
    {"input_name": "John Smith", "variation": "جون سميث", "expected": 1},  # Exact transliteration
    {"input_name": "Mohammed Ali", "variation": "م. علي", "expected": 1},  # Initial variation
    {"input_name": "Anna Johnson", "variation": "أ. جونسون", "expected": 1},  # Initial variation
    {"input_name": "Robert Williams", "variation": "روب وليامز", "expected": 1},  # Nickname variation
    {"input_name": "Elizabeth Taylor", "variation": "ليز تايلور", "expected": 1},  # Nickname variation

    # Negative examples (should predict 0)
    {"input_name": "John Smith", "variation": "مايكل براون", "expected": 0},  # Different name
    {"input_name": "Sarah Connor", "variation": "داود ميلر", "expected": 0},  # Different name
    {"input_name": "Mohammed Ali", "variation": "علي محمد", "expected": 0},  # Reversed name
    {"input_name": "Anna Johnson", "variation": "جونسون آنا", "expected": 0},  # Reversed name
    {"input_name": "Robert Williams", "variation": "وليامز روبرت", "expected": 0},  # Reversed name

    # Edge cases
    {"input_name": " leila alkalba", "variation": "ليلا الكلبة", "expected": 1},  # Exact transliteration
    {"input_name": " mohammed garci", "variation": "محمود غارسي", "expected": 0},  # Initial with transliterated family name
    {"input_name": " مستر كلب", "variation": " ميستر كلب", "expected": 1},  # Initial with transliterated family name
     {"input_name": " محمد", "variation": " أحمد ", "expected":0},
]#,عائشة حسن,arab,1


print("\nRunning inference on test samples...\n")
print(f"{'Input Name':<20} | {'Variation':<20} | {'Prediction':<10} | {'Expected':<10} | {'Confidence':<10} | {'Correct'}")

correct = 0
for sample in test_samples:
    text = f"{sample['input_name']} [SEP] {sample['variation']}"
    pred, confidence = predict(text, model, tokenizer, device)
    is_correct = pred == sample['expected']
    if is_correct:
        correct += 1

    print(f"{sample['input_name']:<20} | {sample['variation']:<20} | {pred:<10} | {sample['expected']:<10} | {confidence:.4f}    | {is_correct}")

accuracy = correct / len(test_samples)
print(f"\nTest Accuracy: {accuracy:.2%} ({correct}/{len(test_samples)})")

# Additional evaluation on validation set
print("\nRunning evaluation on validation set...")
val_results = trainer.evaluate(val_dataset)
print("\nValidation results:")
for key, value in val_results.items():
    print(f"{key}: {value:.4f}")


Running inference on test samples...

Input Name           | Variation            | Prediction | Expected   | Confidence | Correct
John Smith           | جون سميث             | 1          | 1          | 0.7936    | True
Mohammed Ali         | م. علي               | 0          | 1          | 0.9669    | False
Anna Johnson         | أ. جونسون            | 0          | 1          | 0.9563    | False
Robert Williams      | روب وليامز           | 1          | 1          | 0.9603    | True
Elizabeth Taylor     | ليز تايلور           | 1          | 1          | 0.9573    | True
John Smith           | مايكل براون          | 1          | 0          | 0.8579    | False
Sarah Connor         | داود ميلر            | 1          | 0          | 0.9505    | False
Mohammed Ali         | علي محمد             | 1          | 0          | 0.9496    | False
Anna Johnson         | جونسون آنا           | 1          | 0          | 0.9563    | False
Robert Williams      | وليامز روبرت         | 1          | 0 

/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)



Validation results:
eval_loss: 0.4155
eval_accuracy: 0.8763
eval_f1: 0.8787
eval_precision: 0.8645
eval_recall: 0.8933
eval_runtime: 0.7246
eval_samples_per_second: 412.6440
eval_steps_per_second: 13.8010
epoch: 5.0000
